In [1]:
import pandas as pd
import json
import os

In [16]:
def load_event_logs(directory):
    events = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                events.append(json.load(file))
    return events

def apply_events(events):
    records = []
    for event in events:
        record = {'id': event['id'], 'timestamp': event['ts'], 'operation': 'create' if event['op'] == 'c' else 'update'}
        if event['op'] == 'c':
            record.update(event['data'])
        elif event['op'] == 'u':
            record.update(event['set'])
        records.append(record)
    return pd.DataFrame(records)

In [17]:
# Load event logs
accounts_events = load_event_logs('data/accounts')
cards_events = load_event_logs('data/cards')
savings_accounts_events = load_event_logs('data/savings_accounts')

# Apply events to create historical tables
accounts_df = apply_events(accounts_events)
cards_df = apply_events(cards_events)
savings_accounts_df = apply_events(savings_accounts_events)

# Print historical tables
print("Accounts Table:")
print(accounts_df)
print("\nCards Table:")
print(cards_df)
print("\nSavings Accounts Table:")
print(savings_accounts_df)

Accounts Table:
           id      timestamp operation account_id     name   address  \
0  a1globalid  1577863800000    create         a1  Anthony  New York   
1  a1globalid  1577865600000    update        NaN      NaN       NaN   
2  a1globalid  1577890800000    update        NaN      NaN       NaN   
3  a1globalid  1577894400000    update        NaN      NaN   Jakarta   
4  a1globalid  1577926800000    update        NaN      NaN       NaN   
5  a1globalid  1579078860000    update        NaN      NaN       NaN   
6  a1globalid  1579163400000    update        NaN      NaN       NaN   

  phone_number                    email savings_account_id card_id  
0     12345678     anthony@somebank.com                NaN     NaN  
1     87654321                      NaN                NaN     NaN  
2          NaN                      NaN                sa1     NaN  
3          NaN  anthony@anotherbank.com                NaN     NaN  
4          NaN                      NaN                NaN    

In [18]:
accounts_df

,id,timestamp,operation,account_id,name,address,phone_number,email,savings_account_id,card_id
0,a1globalid,1577863800000,create,a1,Anthony,New York,12345678,anthony@somebank.com,NaN,NaN
1,a1globalid,1577865600000,update,NaN,NaN,NaN,87654321,NaN,NaN,NaN
2,a1globalid,1577890800000,update,NaN,NaN,NaN,NaN,NaN,sa1,NaN
3,a1globalid,1577894400000,update,NaN,NaN,Jakarta,NaN,anthony@anotherbank.com,NaN,NaN
4,a1globalid,1577926800000,update,NaN,NaN,NaN,NaN,NaN,NaN,c1
5,a1globalid,1579078860000,update,NaN,NaN,NaN,NaN,NaN,NaN,
6,a1globalid,1579163400000,update,NaN,NaN,NaN,NaN,NaN,NaN,c2


In [8]:
cards_df

,id,timestamp,operation,card_id,card_number,credit_used,monthly_limit,status
0,c1globalid,1577926800000,create,c1,11112222,0.0,30000.0,PENDING
1,c1globalid,1578159000000,update,NaN,NaN,NaN,NaN,ACTIVE
2,c1globalid,1578313800000,update,NaN,NaN,12000.0,NaN,NaN
3,c1globalid,1578420000000,update,NaN,NaN,19000.0,NaN,NaN
4,c1globalid,1578654000000,update,NaN,NaN,0.0,NaN,NaN
5,c1globalid,1579078800000,update,NaN,NaN,NaN,NaN,CLOSED
6,c2globalid,1579163400000,create,c2,12123434,0.0,70000.0,PENDING
7,c2globalid,1579298400000,update,NaN,NaN,NaN,NaN,ACTIVE
8,c2globalid,1579361400000,update,NaN,NaN,37000.0,NaN,NaN


In [9]:
savings_accounts_df

,id,timestamp,operation,savings_account_id,balance,interest_rate_percent,status
0,sa1globalid,1577890800000,create,sa1,0.0,1.5,ACTIVE
1,sa1globalid,1577955600000,update,NaN,15000.0,NaN,NaN
2,sa1globalid,1578159060000,update,NaN,NaN,3.0,NaN
3,sa1globalid,1578648600000,update,NaN,40000.0,NaN,NaN
4,sa1globalid,1578654000000,update,NaN,21000.0,NaN,NaN
5,sa1globalid,1579078860000,update,NaN,NaN,1.5,NaN
6,sa1globalid,1579298460000,update,NaN,NaN,4.0,NaN
7,sa1globalid,1579505400000,update,NaN,33000.0,NaN,NaN


In [20]:
import os
import json
import pandas as pd
from glob import glob

def load_events(directory):
    events = []
    for filepath in glob(os.path.join(directory, '*.json')):
        with open(filepath, 'r') as file:
            events.append(json.load(file))
    return events

def process_events(events):
    records = {}
    for event in sorted(events, key=lambda x: x['ts']):
        record_id = event['id']
        if event['op'] == 'c':
            records[record_id] = event['data']
        elif event['op'] == 'u':
            if record_id in records:
                records[record_id].update(event['set'])
    return records

def load_and_process_table(directory):
    events = load_events(directory)
    records = process_events(events)
    return pd.DataFrame.from_dict(records, orient='index')

#def main():
# Load and process each table
accounts_df = load_and_process_table('data/accounts')
cards_df = load_and_process_table('data/cards')
savings_accounts_df = load_and_process_table('data/savings_accounts')

# Display the historical view of each table
print("Accounts Table:")
print(accounts_df)

print("\nCards Table:")
print(cards_df)

print("\nSaving Accounts Table:")
print(savings_accounts_df)

# Join the tables to get a denormalized view
joined_df = accounts_df.merge(cards_df, left_on='card_id', right_on='card_id', how='left', suffixes=('_account', '_card'))
joined_df = joined_df.merge(savings_accounts_df, left_on='savings_account_id', right_on='savings_account_id', how='left', suffixes=('', '_saving_account'))

print("\nJoined Table:")
print(joined_df)

# Identify transactions (changes in balance or credit used)
transactions = []
for event in load_events('data/cards'):
    if event['op'] == 'u' and 'credit_used' in event['set']:
        transactions.append({'id': event['id'], 'ts': event['ts'], 'type': 'credit_used', 'value': event['set']['credit_used']})

for event in load_events('data/savings_accounts'):
    if event['op'] == 'u' and 'balance' in event['set']:
        transactions.append({'id': event['id'], 'ts': event['ts'], 'type': 'balance', 'value': event['set']['balance']})

transactions_df = pd.DataFrame(transactions)

print("\nTransactions:")
print(transactions_df)

# if __name__ == "__main__":
#     main()


Accounts Table:
           account_id     name  address phone_number                    email  \
a1globalid         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com   

           savings_account_id card_id  
a1globalid                sa1      c2  

Cards Table:
           card_id card_number  credit_used  monthly_limit  status
c1globalid      c1    11112222            0          30000  CLOSED
c2globalid      c2    12123434        37000          70000  ACTIVE

Saving Accounts Table:
            savings_account_id  balance  interest_rate_percent  status
sa1globalid                sa1    33000                    4.0  ACTIVE

Joined Table:
  account_id     name  address phone_number                    email  \
0         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com   

  savings_account_id card_id card_number  credit_used  monthly_limit  status  \
0                sa1      c2    12123434        37000          70000  ACTIVE   

   balance  interest_rate_percent statu

In [10]:
accounts_df

,account_id,name,address,phone_number,email,savings_account_id,card_id
a1globalid,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c2


In [11]:
cards_df

,card_id,card_number,credit_used,monthly_limit,status
c1globalid,c1,11112222,0,30000,CLOSED
c2globalid,c2,12123434,37000,70000,ACTIVE


In [12]:
savings_accounts_df

,savings_account_id,balance,interest_rate_percent,status
sa1globalid,sa1,33000,4.0,ACTIVE


In [13]:
joined_df

,account_id,name,address,phone_number,email,savings_account_id,card_id,card_number,credit_used,monthly_limit,status,balance,interest_rate_percent,status_saving_account
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c2,12123434,37000,70000,ACTIVE,33000,4.0,ACTIVE


In [14]:
transactions_df

,id,ts,type,value
0,c1globalid,1578313800000,credit_used,12000
1,c1globalid,1578420000000,credit_used,19000
2,c1globalid,1578654000000,credit_used,0
3,c2globalid,1579361400000,credit_used,37000
4,sa1globalid,1577955600000,balance,15000
5,sa1globalid,1578648600000,balance,40000
6,sa1globalid,1578654000000,balance,21000
7,sa1globalid,1579505400000,balance,33000
